# Lab / Lab Manager Tutorial

The lab environment serves as an environment for a user to conduct chemical experiments as they would in a physical lab, further the lab environment provides the opportunity to train a highlevel agent to synthesize materials using the 4 lab benches and tha gents associated with each. The environment allows the user to use a variety of reward functions based on the cost of real world lab equipment, material costs purity and labour costs. In this tutorial we will walk through how a lab manager agent would walk through the process of trying to synthesize dodecane. Further we will walk through the manager wrapper that we have developed which gives a simple api for an agent to run in an environment.

## Lab:

First off we can go over the 4 lab benches that the agent will have access to when trying to synthesize a material.

| Bench Index: | Bench Name:            | Bench Description:                                                                                                                                                                                                                                                                                                                                                            |
|--------------|------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 0            | Reaction Bench         | The reaction bench serves as a tool for an  agent to operate within an environment where  they can learn how to optimize the process by  which they synthesize a material. By using a  predefined reaction, we simulate the reaction  allowing the agent to learn how best to  optimize the reaction.                                                                         |
| 1            | Extraction Bench       | The extraction bench serves as a tool for an  agent to learn how to extract materials from  one solution to another.                                                                                                                                                                                                                                                          |
| 2            | Distillation Bench     | The distillation bench serves as a tool for  an agent to learn how to distill and purify  solutions down to a single desired material.                                                                                                                                                                                                                                        |
| 3            | Characterization Bench | The characterization bench is different from  the rest of the benches as it doesn't have an agent trained to run in it. In this case the  characterization bench takes in a vessel and  a desired analysis technique and returns the  results of that analysis, for instance  performing an absorbtion spectra analysis  and returning the spectral graph back to the  agent. |

In [ ]:
import chemistrylab
import numpy as np

In [ ]:
from chemistrylab.lab.lab import Lab

In [ ]:
lab = Lab()
# Notice how there are currently no vessels in our shelf
lab.shelf.vessels

Now that we have initialized the lab environmet we should take a look at the action space for the lab environment.

| Bench Index | Bench Env Index | Vessel Index      | Agent Index      |
|-------------|-----------------|-------------------|------------------|
| 0 - 4         | 0 - Max_Num_Env   | 0 - Max_Num_Vessels | 0 - Max_Num_Agents |

So part of the challenge for an agent running in this environment will be that each bench has a different number of bench environments, and available agents. For instance the user could have 10 unique reactions which only require a general extraction and a general distillation, in this case Max_Num_Env will be 10 even if the extraction bench is selected and even though there is only 1 registered environment for the extraction bench. As such if the user selects an agent or bench environment that are not available for a certain bench the agent will recieve a negative reward. Now that we have looked over the action space it will help to look over the lab environment in some examples that will give some more detail.

First let's take a look at all the environemnts registered to each bench:

In [ ]:
# All reaction environments that are available to the agent
lab.reactions

In [ ]:
# All extraction environments that are available to the agent
lab.extractions

In [ ]:
# All distillations environments that are available to the agent
lab.distillations

Now that we know what environments are registered let's take a look at the available agents:

In [ ]:
# All reaction agents
lab.react_agents

In [ ]:
# All extraction agents
lab.extract_agents

In [ ]:
# All distillation agents
lab.distill_agents

Perfect, now that we can understand our action space, let's perform some actions!

In [ ]:
react_action = np.array([0, 0, 0, 0])
lab.step(react_action)

From above we see that the react_action is loading the reaction bench with the WurtzReact-v1 environment, the 0th vessel and a random agent. From the output we see the following: ((reward, analysis_array), Done). Now that we have run a step, let's take a look at the vessels available to the lab:

In [ ]:
lab.shelf.vessels

In [ ]:
lab.shelf.vessels[0].get_material_dict()

notice how we now have a vessel in the shelf and when we look at it we can see chemicals from the wurtz reaction. Now that we have these leftover in our vessel, we want to try and extract dodecane out of the vessel.

In [ ]:
extract_action = np.array([1, 0, 0, 0])
lab.step(extract_action)

From above we see that the extract_action is loading the extraction bench with the 'WurtzExtract-v1' environment, the 0th vessel (as seen above) and a random agent

In [ ]:
lab.shelf.vessels

In [ ]:
for vessel in lab.shelf.vessels:
    print(vessel.get_material_dict())
    print("_____________")

From the above we can clearly see that there are 2 new vessels that have been added to our shelf courtesy of the extraction bench.

In [ ]:
distill_action = np.array([2, 0, 0, 0])
lab.step(distill_action)

From above we see that the distill_action is loading the distillation bench with the 'Distillation-v0' environment, the 0th vessel (as seen above) and a random agent

In [ ]:
for vessel in lab.shelf.vessels:
    print(vessel.get_material_dict())
    print("_____________")

In [ ]:
analysis_action = np.array([3, 0, 0, 0])
lab.step(analysis_action)

Lastly we use the characterization bench. In this case we are going to perform an absobtion spectra analysis on our vessel that we get back. This is designed so that the agent can identify what is in the vessel without explicitly telling the agent. That's all for this part of the tutorial on the Lab environment, next we will cover the Lab Manager wrapper for the lab environment. 

## Lab Manager:

The Lab Manager at the moment doesn't support training agents, however it does support pre trained or heuristic agents, or even a human agent.

In [ ]:
from chemistrylab.lab.manager import Manager

In [ ]:
manager = Manager()
manager.agents

The output above shows us what agents are available to run the Manager environment. You can also make your own custom agents using our agent api.

In [ ]:
from chemistrylab.lab.agent import Agent

In [ ]:
class CustomAgent(Agent):
    def __init__(self):
        self.name = 'custom_agent'
        self.prev_actions = []

    def run_step(self, env, spectra):
        """
        this is the function where the operation of your model or heuristic agent is defined
        """
        action = np.array([])
        if len(self.prev_actions) == 0:
            action = np.array([0, 0, 0, 0])
        elif self.prev_actions[-1][0] == 0:
            action = np.array([1, 0, 0, 0])
        elif self.prev_actions[-1][0] == 1:
            action = np.array([2, 0, 2, 0])
        elif self.prev_actions[-1][0] == 2:
            action = np.array([3, 0, 2, 0])
        elif self.prev_actions[-1][0] == 3:
            action = np.array([4, 0, 0, 0])
        else:
            assert False
        self.prev_actions.append(action)
        return action

In [ ]:
custom_agent = CustomAgent

In [ ]:
# Now that we have created a custom agent to run the whole lab process for us we need to register it
# with our environment
manager.register_agent('custom_agent', custom_agent)

In [ ]:
# Now that the agent has been registered we change the mode to the name of the new agent so we will run the manager
# with the new agent
manager.mode = 'custom_agent'

In [ ]:
manager.run()

### Closing Remarks:
And the api is just as simple as that! We hope this has been informative and you should now be able to run the Lab and Lab Manager smoothly.